In [1]:
import cv2
import numpy as np

# Load the image
path='michico.jpg'
image=cv2.imread('michico.jpg')

# Original size of the image
height, width, channels = image.shape
print(f"Width: {width}, Height: {height}")

Width: 1280, Height: 1280


In [2]:
# Check if the image was loaded correctly
if image is None:
    print ("Error:please check the path.")
    exit()

In [3]:
# Resize the image
new_width=512
new_height=512
resized_image=cv2.resize(image,(new_width,new_height))

In [4]:
# Resize the image
new_width=512
new_height=512
resized_image=cv2.resize(image,(new_width,new_height))

In [5]:
# Crop the image
def crop_circle (image, radius_factor=0.85):
    height,width=image.shape[:2]
    mask=np.zeros((height,width), dtype=np.uint8)
    center=(int(width/2), int(height/2))
    original_radius= min(center[0], center[1])
    radius=int(original_radius*radius_factor)
    cv2.circle(mask,center,radius,(255,),-1, cv2.LINE_AA)
    masked_image=cv2.bitwise_and(image,image,mask=mask)
    return masked_image

cropped_image=crop_circle(resized_image, radius_factor=0.6)
cv2.imwrite('cropped.jpg', cropped_image)

True

In [6]:
# Convert the cropped image to grayscale
gray_scaled_image = cv2.cvtColor(cropped_image, cv2.COLOR_BGR2GRAY)

In [7]:
# Normalize the image 
normalized_image= gray_scaled_image/255.0

In [8]:
# Convert normalized image back to 8-bit format for histogram equalization
normalized_image_8bit = (normalized_image * 255).astype(np.uint8)

# Apply histogram equalization
equalized_image = cv2.equalizeHist(normalized_image_8bit)

In [9]:
# Read the blurred image
blurred_image = cv2.GaussianBlur(normalized_image, (3, 3), 0)

In [10]:
# Apply erosion and dilation to separate fungi colonies
eroded_image = cv2.erode(blurred_image, np.ones((3, 3), np.uint8), iterations=1)
dilated_image = cv2.dilate(eroded_image, np.ones((3, 3), np.uint8), iterations=1)

In [11]:
# Perform edge detection on the blurred image
# Convert the normalized image back to an 8-bit image
blurred_image_8bit = (blurred_image * 255).astype(np.uint8)
edges = cv2.Canny(blurred_image_8bit, 50, 100)


In [12]:
# Adaptive Thresholding
adaptive_thresh_image = cv2.adaptiveThreshold(blurred_image_8bit, 255,
                                              cv2.ADAPTIVE_THRESH_GAUSSIAN_C,
                                              cv2.THRESH_BINARY_INV, 11, 2)

# Otsu's Binarization
_, otsu_thresh_image = cv2.threshold(blurred_image_8bit, 0, 255, 
                                     cv2.THRESH_BINARY_INV + cv2.THRESH_OTSU)




In [13]:
# Apply morphological operations to clean up the binary image
kernel = np.ones((3, 3), np.uint8)
cleaned_image = cv2.morphologyEx(adaptive_thresh_image, cv2.MORPH_CLOSE, kernel, iterations=2)

In [14]:
# Find contours
contours, _ = cv2.findContours(cleaned_image, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

# Draw contours on the original image
contour_image = cropped_image.copy()
cv2.drawContours(contour_image, contours, -1, (0, 255, 0), 2)

array([[[0, 0, 0],
        [0, 0, 0],
        [0, 0, 0],
        ...,
        [0, 0, 0],
        [0, 0, 0],
        [0, 0, 0]],

       [[0, 0, 0],
        [0, 0, 0],
        [0, 0, 0],
        ...,
        [0, 0, 0],
        [0, 0, 0],
        [0, 0, 0]],

       [[0, 0, 0],
        [0, 0, 0],
        [0, 0, 0],
        ...,
        [0, 0, 0],
        [0, 0, 0],
        [0, 0, 0]],

       ...,

       [[0, 0, 0],
        [0, 0, 0],
        [0, 0, 0],
        ...,
        [0, 0, 0],
        [0, 0, 0],
        [0, 0, 0]],

       [[0, 0, 0],
        [0, 0, 0],
        [0, 0, 0],
        ...,
        [0, 0, 0],
        [0, 0, 0],
        [0, 0, 0]],

       [[0, 0, 0],
        [0, 0, 0],
        [0, 0, 0],
        ...,
        [0, 0, 0],
        [0, 0, 0],
        [0, 0, 0]]], dtype=uint8)

In [15]:
# Calculate the percentage of area covered by fungi colonies
# Count the number of white pixels in the cleaned binary image
fungi_area = np.sum(cleaned_image == 255)
# Calculate the total number of pixels in the image
total_area = cleaned_image.size
# Calculate the percentage
fungi_percentage = (fungi_area / total_area) * 100
print(f"Percentage of area covered by fungi colonies: {fungi_percentage:.2f}%")

Percentage of area covered by fungi colonies: 25.70%


In [16]:
# Display the images
cv2.imshow('Original Image', image)
cv2.imshow('Edges', edges)
cv2.imshow('Adaptive Threshold Image', adaptive_thresh_image)
cv2.imshow('Otsu Threshold Image', otsu_thresh_image)
cv2.imshow('Cleaned Image', cleaned_image)
cv2.imshow('Contour Image', contour_image)


cv2.waitKey(0)
cv2.destroyAllWindows()